In [1]:
datatype = "SimBLT"

In [2]:
import os
import glob
import pandas as pd
import gzip

# Path to the truthset file
truthset_path = (
    "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/3.PU_BLT/"
    "Step1_Validated_union_snv_VAF_RGN.VAF_added.txt"
)

# List of directories containing VCF files
vcf_directories = [
    #"/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/AdxBLT/SNV",
    "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/"+ datatype +"/SNV",
]

# Mapping of directory names to truthset file columns
# This is for getting the truth set VAF from pileup data. It is not critical for
# generating accuracy score, made for internal checking.
directory_column_mapping = {
    "BCM": "bcm_colo829blt50_illumina_400x",
    "BROAD": "broad_colo829blt50_illumina_160x",
    "NYGC": "nygc_colo829blt50_illumina_200X",
    "UW": "uw_colo829blt50_illumina_350x",
    "WASHU": "washu_colo829blt50_illumina_500x",
    "100X": "insilico100X",
    "200X": "insilico200X",
    "300X": "insilico300X",
    "400X": "insilico400X",
    "500X": "insilico500X",
}

# Load the truthset file
truthset_df = pd.read_csv(truthset_path, sep="\t")
truthset_df["CHROM_POS_REF_ALT"] = truthset_df["#variant"].str.replace(":", "_")


# -------------------------------
# Helper functions
# -------------------------------

def extract_af(format_field, sample_field):
    """Extract AF value from FORMAT/SAMPLE fields in VCF files."""
    if format_field == "AF":  # special case: FORMAT is just AF
        try:
            return float(sample_field)
        except ValueError:
            return None

    format_keys = format_field.split(":")
    sample_values = sample_field.split(":")
    if "AF" in format_keys:
        af_index = format_keys.index("AF")
        try:
            return float(sample_values[af_index])
        except (IndexError, ValueError):
            return None
    return None


def extract_hd_af(sample_field):
    """Extract HD_AF value (last field) from SAMPLE field in Utah files."""
    sample_values = sample_field.split(":")
    af_value = sample_values[-1]  # The last value
    if af_value != ".":
        return round(float(af_value), 3)
    return None


def extract_af_nygc(format_field, sample_field):
    """Extract AF from NYGC files using VAF field."""
    format_keys = format_field.split(":")
    sample_values = sample_field.split(":")
    af_index = format_keys.index("VAF") if "VAF" in format_keys else None
    if af_index is not None:
        return float(sample_values[af_index])
    return None


def process_vcf_files_in_directory(subdirectory, truth_column):
    subdirectory_path = os.path.join(vcf_directory, subdirectory)
    file_paths = glob.glob(os.path.join(subdirectory_path, "*.vcf.gz"))

    for filename in os.listdir(subdirectory_path):
        if filename.endswith(".vcf.gz") and "NC" not in filename:
            vcf_path = os.path.join(subdirectory_path, filename)

            try:
                # Load the VCF header
                with gzip.open(vcf_path, "rt") as f:
                    headers = []
                    for line in f:
                        if line.startswith("#"):
                            if line.startswith("#CHROM"):
                                headers = line.strip().split("\t")
                            continue
                        else:
                            break

                if not headers:
                    raise ValueError(f"Header not found in file: {filename}")

                # Load the VCF content
                vcf_df = pd.read_csv(
                    vcf_path, sep="\t", comment="#", header=None, dtype=str
                )
                if len(vcf_df.columns) != len(headers):
                    raise ValueError(f"Column length mismatch in file: {filename}")

                vcf_df.columns = headers
                vcf_df["CHROM_POS_REF_ALT"] = (
                    vcf_df["#CHROM"] + "_" +
                    vcf_df["POS"] + "_" +
                    vcf_df["REF"] + "_" +
                    vcf_df["ALT"]
                )
                sample_column = headers[headers.index("FORMAT") + 1]

                
                # Extract VAF depending on file origin
                if "Utah" in filename:
                    vcf_df["VAF"] = vcf_df[sample_column].apply(extract_hd_af)
                elif "M5_NYGC" in filename: #For lancet
                    vcf_df["VAF"] = vcf_df.apply(
                        lambda row: extract_af_nygc(row["FORMAT"], row[sample_column]),
                        axis=1,
                    )
                else:
                    vcf_df["VAF"] = vcf_df.apply(
                        lambda row: extract_af(row["FORMAT"], row[sample_column]),
                        axis=1,
                    )

                # Merge with truthset
                common_variants = pd.merge(
                    vcf_df,
                    truthset_df,
                    on="CHROM_POS_REF_ALT",
                    how="left",
                    suffixes=("", "_truth"),
                )

                # Build output DataFrame
                output_df = pd.DataFrame({
                    "Chrom": common_variants["#CHROM"],
                    "pos": common_variants["POS"],
                    "ref": common_variants["REF"],
                    "alt": common_variants["ALT"],
                    "TS_VAF": common_variants[truth_column].fillna("NA"),
                    "VAF": common_variants["VAF"],
                    "Category": common_variants.apply(
                        lambda row: "TP" if pd.notna(row[truth_column]) else "Masked",
                        axis=1,
                    ),
                })

                # Save output
                output_path = vcf_path.replace(".vcf.gz", ".parsed.tsv")
                output_df.to_csv(output_path, sep="\t", index=False)

                # Print summary for quick check
                print(f"Output for {filename}:")
                print(output_df.head())

            except Exception as e:
                print(f"Error processing file {filename}: {e}")


# -------------------------------
# Main loop
# -------------------------------
for vcf_directory in vcf_directories:
    for subdirectory in os.listdir(vcf_directory):
        if (
            os.path.isdir(os.path.join(vcf_directory, subdirectory))
            and subdirectory in directory_column_mapping
        ):
            truth_column = directory_column_mapping[subdirectory]
            print(subdirectory, vcf_directory, truth_column)
            process_vcf_files_in_directory(subdirectory, truth_column)
    break


100X /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV insilico100X
Output for M10_Utah_insilico_BLT50_100X.norm.snps.vcf.gz:
  Chrom       pos ref alt    TS_VAF    VAF Category
0  chr1   1182262   C   T  0.057377  0.078       TP
1  chr1   1321993   G   A  0.046358  0.040       TP
2  chr1   3436651   C   A   0.04386  0.051       TP
3  chr1   4320668   C   A  0.044643  0.053       TP
4  chr1  10924788   G   A  0.041322  0.045       TP
Output for M11_Utah_insilico_BLT50_100X.norm.snps.vcf.gz:
  Chrom       pos ref alt    TS_VAF    VAF Category
0  chr1   1182262   C   T  0.057377  0.078       TP
1  chr1   1321993   G   A  0.046358  0.040       TP
2  chr1   3436651   C   A   0.04386  0.051       TP
3  chr1   4320668   C   A  0.044643  0.053       TP
4  chr1  10924788   G   A  0.041322  0.045       TP
Output for M12_Utah_insilico_BLT50_100X.norm.snps.vcf.gz:
  Chrom       pos ref alt    TS_VAF    VAF Category
0  chr1   1182262   C   T  0.057377  0.078   

# Run /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/1.benchmark/workflow/Annotate.sh

In [3]:
import os
import glob
import pandas as pd

# Base directory path
base_dir = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/"+ datatype +"/SNV"

# Function to extract AF value from FORMAT field in VCF files
def extract_af(format_field, sample_field):
    format_keys = format_field.split(':')
    sample_values = sample_field.split(':')
    af_index = format_keys.index('AF') if 'AF' in format_keys else None
    if af_index is not None:
        return round(float(sample_values[af_index]), 3)
    return None

# Function to extract HD_AF value from SAMPLE field in Utah files
def extract_hd_af(sample_field):
    sample_values = sample_field.split(':')
    hd_af_value = sample_values[-1]  # The last value
    if hd_af_value != '.':
        return round(float(hd_af_value), 3)
    return None

# Function to extract AD value from FORMAT field and calculate VAF (NYGC files)
def extract_nygc_vaf(format_field, sample_field):
    format_keys = format_field.split(':')
    sample_values = sample_field.split(':')
    ad_index = format_keys.index('AD') if 'AD' in format_keys else None
    if ad_index is not None:
        ad_value = sample_values[ad_index]
        ref_allele, alt_allele = map(float, ad_value.split(','))
        dp = ref_allele + alt_allele
        if dp > 0:
            return round(alt_allele / dp, 3)
    return None

# Read VCF file into a dictionary (ignore headers)
def read_vcf_to_dict(vcf_path, value_field='AF'):
    data = {}
    with open(vcf_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            parts = line.strip().split('\t')
            chrom = parts[0]
            pos = int(parts[1])
            ref = parts[3]
            alt = parts[4]
            format_field = parts[8]
            sample_field = parts[9]
            if vcf_path.startswith('NYGC'):
                af_value = extract_nygc_vaf(format_field, sample_field)
            elif value_field == 'AF':
                af_value = extract_af(format_field, sample_field)
            else:
                af_value = extract_hd_af(sample_field)
            key = (chrom, pos, ref, alt)
            data[key] = af_value
    return data

# Process all parsed.tsv files
parsed_files = glob.glob(os.path.join(base_dir, '**', '*.parsed.tsv'), recursive=True)

for parsed_tsv_path in parsed_files:
    # Generate VCF file paths
    prefix = parsed_tsv_path.replace('.parsed.tsv', '')
    print (parsed_tsv_path, prefix)
    germline_vcf_path = prefix + '.vcf.gz.NC.germline.vcf'
    nonvariant_vcf_path = prefix + '.vcf.gz.NC.nonvariant.vcf'
    output_parsed2_tsv_path = prefix + '.parsed2.tsv'
    
    # Read VCF files
    germline_dict = read_vcf_to_dict(germline_vcf_path, 'AF' if 'Utah' not in germline_vcf_path else 'HD_AF')
    nonvariant_dict = read_vcf_to_dict(nonvariant_vcf_path, 'AF' if 'Utah' not in nonvariant_vcf_path else 'HD_AF')

    # Read the TSV file
    parsed_df = pd.read_csv(parsed_tsv_path, sep='\t')

    # Standardize the format of the 'Chrom' column by converting 'Chrom' and 'pos' columns to integer
    parsed_df['pos'] = parsed_df['pos'].astype(int)

    # List to store the results
    result_rows = []

    # Update and merge data
    for _, row in parsed_df.iterrows():
        key = (row['Chrom'], row['pos'], row['ref'], row['alt'])
        original_category = row['Category']
        
        if original_category == 'TP' and (key in germline_dict or key in nonvariant_dict):
            print(f"TP row found: {row}")
        else:
            if key in germline_dict:
                row['Category'] = 'FP:Germline'
                row['VAF'] = germline_dict[key]
            elif key in nonvariant_dict:
                row['Category'] = 'FP:Nonvariant'
                row['VAF'] = nonvariant_dict[key]
        
        result_rows.append(row)

    # Convert results to DataFrame
    result_df = pd.DataFrame(result_rows)

    # Save the results
    result_df.to_csv(output_parsed2_tsv_path, sep='\t', index=False)

    print(f"Updated file saved to {output_parsed2_tsv_path}")


/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M10_Utah_insilico_BLT50_100X.norm.snps.parsed.tsv /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M10_Utah_insilico_BLT50_100X.norm.snps
Updated file saved to /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M10_Utah_insilico_BLT50_100X.norm.snps.parsed2.tsv
/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M11_Utah_insilico_BLT50_100X.norm.snps.parsed.tsv /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M11_Utah_insilico_BLT50_100X.norm.snps
Updated file saved to /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M11_Utah_insilico_BLT50_100X.norm.snps.parsed2.tsv
/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M12_Utah_insilico_BLT50_100X.nor

# Update the TP with in vitro mutations

In [4]:
import os
import glob
import pandas as pd

# --- Paths ---
base_dir = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/"+ datatype +"/SNV"
invitro_truth_path = (
    "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/4.Invitro/input/COLO829BLT.finalInVitro.snps.sorted_VAF_RGN.VAF_added.txt"
)

# --- Load in vitro truth set ---
invitro_df = pd.read_csv(invitro_truth_path, sep="\t")
# Normalize key
invitro_df["CHROM_POS_REF_ALT"] = invitro_df["#variant"].str.replace(":", "_")
invitro_map = dict(
    zip(invitro_df["CHROM_POS_REF_ALT"], invitro_df["VAF_Ill"])
)

# --- Process parsed2.tsv files ---
parsed2_files = glob.glob(os.path.join(base_dir, "**", "*.parsed2.tsv"), recursive=True)

for parsed2_path in parsed2_files:
    parsed3_path = parsed2_path.replace(".parsed2.tsv", ".parsed3.tsv")

    print(f"Processing {parsed2_path} → {parsed3_path}")

    df = pd.read_csv(parsed2_path, sep="\t")

    # Build variant key consistent with invitro truth set
    df["CHROM_POS_REF_ALT"] = (
        df["Chrom"].astype(str)
        + "_"
        + df["pos"].astype(str)
        + "_"
        + df["ref"].astype(str)
        + "_"
        + df["alt"].astype(str)
    )

    # Update if present in invitro set
    def update_row(row):
        key = row["CHROM_POS_REF_ALT"]
        if key in invitro_map:
            row["TS_VAF"] = row["VAF"]
            row["Category"] = "TP_invitro"
            #print (row)
        return row

    df = df.apply(update_row, axis=1)

    # Drop helper column
    df = df.drop(columns=["CHROM_POS_REF_ALT"])

    # Save updated file
    df.to_csv(parsed3_path, sep="\t", index=False)

    print(f"✅ Updated file saved: {parsed3_path}")


Processing /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M10_Utah_insilico_BLT50_100X.norm.snps.parsed2.tsv → /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M10_Utah_insilico_BLT50_100X.norm.snps.parsed3.tsv
✅ Updated file saved: /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M10_Utah_insilico_BLT50_100X.norm.snps.parsed3.tsv
Processing /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M11_Utah_insilico_BLT50_100X.norm.snps.parsed2.tsv → /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M11_Utah_insilico_BLT50_100X.norm.snps.parsed3.tsv
✅ Updated file saved: /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/SimBLT/SNV/100X/M11_Utah_insilico_BLT50_100X.norm.snps.parsed3.tsv
Processing /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/

In [5]:
import pandas as pd
import os
import glob


def create_combined_matrix(first_data_path, truth_data, match_columns, truth_column):
    # Load the first file
    first_data = pd.read_csv(first_data_path, sep='\t')

    # --- Clean the truth column ---
    if truth_column in truth_data.columns:
        truth_data[truth_column] = truth_data[truth_column].replace(['.', 'NA'], 0)
        truth_data[truth_column] = pd.to_numeric(truth_data[truth_column], errors="coerce").fillna(0)
    else:
        raise ValueError(f"Truth column {truth_column} not found in truth_data")

    # --- Clean numeric columns in first_data ---
    for col in ["VAF"]:
        if col in first_data.columns:
            first_data[col] = first_data[col].replace(['.', 'NA'], 0)
            first_data[col] = pd.to_numeric(first_data[col], errors="coerce").fillna(0)

    # Define bins
    bins = [0, 0.005, 0.01, 0.015, 0.02, 0.03, 0.04, 0.05, 0.1, 1.0]
    bin_labels = [f'{bins[i]}-{bins[i+1]}' for i in range(len(bins)-1)]

    # Create the truth matrix
    truth_matrix = pd.cut(
        truth_data[truth_column],
        bins=bins,
        labels=bin_labels,
        include_lowest=True,
        right=False
    ).value_counts().sort_index()

    # Filter first data for TP, FP, and Masked categories and matching columns
    tp_data = pd.merge(
        first_data[first_data['Category'].isin(['TP', 'TP_invitro'])],
        truth_data[match_columns],
        on=match_columns
    )

    fp_data = first_data[first_data['Category'].str.contains('FP', na=False)]
    masked_data = first_data[first_data['Category'].str.contains('Masked', na=False)]

    # Create the TP, FP, and Masked matrices
    tp_matrix = pd.cut(tp_data['VAF'], bins=bins, labels=bin_labels, include_lowest=True, right=False).value_counts().sort_index()
    fp_matrix = pd.cut(fp_data['VAF'], bins=bins, labels=bin_labels, include_lowest=True, right=False).value_counts().sort_index()
    masked_matrix = pd.cut(masked_data['VAF'], bins=bins, labels=bin_labels, include_lowest=True, right=False).value_counts().sort_index()

    return truth_matrix, tp_matrix, fp_matrix, masked_matrix, bin_labels




def process_all_files_in_directory(directory, truth_data, match_columns, output_dir):
    # Only process subdirectories that are in directory_column_mapping
    subdirectories = [
        d for d in os.listdir(directory)
        if os.path.isdir(os.path.join(directory, d)) and d in directory_column_mapping
    ]

    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(directory, subdirectory)
        file_paths = glob.glob(os.path.join(subdirectory_path, '*.parsed3.tsv'))

        combined_matrix = None
        truth_column = directory_column_mapping.get(subdirectory, None)

        if truth_column is None:
            print(f"Truth column mapping not found for subdirectory: {subdirectory}")
            continue

        for file_path in file_paths:
            # Extract the filename without extension to use as a column name
            file_name_parts = os.path.splitext(os.path.basename(file_path))[0].split('_')
            file_name = "_".join(file_name_parts[:2])

            # Create the combined matrix for the current file
            truth_matrix, tp_matrix, fp_matrix, masked_matrix, bin_labels = create_combined_matrix(
                file_path, truth_data, match_columns, truth_column
            )

            # Create a DataFrame for the current TP, FP, and Masked Matrices with the filename as the column name
            tp_df = tp_matrix.to_frame(name=f'TP-Matrix-{file_name}')
            fp_df = fp_matrix.to_frame(name=f'FP-Matrix-{file_name}')
            masked_df = masked_matrix.to_frame(name=f'Masked-Matrix-{file_name}')

            if combined_matrix is None:
                # Initialize the combined matrix with the first file's results
                combined_matrix = pd.DataFrame({
                    'Bin': bin_labels,
                    'Truth-Matrix': truth_matrix
                })
                combined_matrix = combined_matrix.join(tp_df, how='outer').join(fp_df, how='outer').join(masked_df, how='outer')
            else:
                # Join the current TP, FP, and Masked Matrix results with the combined matrix
                combined_matrix = combined_matrix.join(tp_df, how='outer').join(fp_df, how='outer').join(masked_df, how='outer')

        if combined_matrix is not None:
            # Calculate Sensitivity, Precision, and F1 Score for each TP Matrix column
            for col in combined_matrix.columns:
                if col.startswith('TP-Matrix'):
                    tp_col = col
                    fp_col = col.replace('TP', 'FP')
                    precision_col = f'Precision-{col.split("-")[2]}'
                    sensitivity_col = f'Sensitivity-{col.split("-")[2]}'
                    f1_score_col = f'F1-Score-{col.split("-")[2]}'

                    combined_matrix[precision_col] = combined_matrix[tp_col] / (combined_matrix[tp_col] + combined_matrix[fp_col])
                    combined_matrix[sensitivity_col] = combined_matrix[tp_col] / combined_matrix['Truth-Matrix']
                    combined_matrix[f1_score_col] = 2 * (combined_matrix[precision_col] * combined_matrix[sensitivity_col]) / (combined_matrix[precision_col] + combined_matrix[sensitivity_col])

            # Fill NaN values with 0
            combined_matrix = combined_matrix.fillna(0)

            # Rename columns to replace spaces with '-' and remove parentheses
            combined_matrix.columns = [col.replace(' ', '-').replace('(', '').replace(')', '') for col in combined_matrix.columns]

            # Print the file used for the combined matrix
            print(f"Processed data for subdirectory: {subdirectory}")

            # Save the combined matrix to a CSV file in the specified output directory
            output_file_name = os.path.basename(subdirectory) + '_truth_matrix.tsv'
            output_path = os.path.join(output_dir, output_file_name)
            combined_matrix.to_csv(output_path, sep='\t', index=False)


# File paths and columns
directory = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/Benchmark/Variants/Challenge_2025/"+ datatype +"/SNV"
truth_data_path1 = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/3.PU_BLT/Step1_Validated_union_snv_VAF_RGN.VAF_added.txt"
truth_data_path2 = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/4.Invitro/input/COLO829BLT.finalInVitro.snps.sorted_VAF_RGN.VAF_added.txt"
match_columns = ['Chrom', 'pos', 'ref', 'alt']
output_dir = directory

# Mapping of directory names to truthset file columns
directory_column_mapping = {
    "BCM": "bcm_colo829blt50_illumina_400x",
    "BROAD": "broad_colo829blt50_illumina_160x",
    "NYGC": "nygc_colo829blt50_illumina_200X",
    "UW": "uw_colo829blt50_illumina_350x",
    "WASHU": "washu_colo829blt50_illumina_500x",
    "100X": "insilico100X",
    "200X": "insilico200X",
    "300X": "insilico300X",
    "400X": "insilico400X",
    "500X": "insilico500X",
}

# --- NEW: load and merge truth sets ---
truth1 = pd.read_csv(truth_data_path1, sep="\t")
truth2 = pd.read_csv(truth_data_path2, sep="\t")

truth1 = truth1.rename(columns={'#variant': 'variant'})
truth2 = truth2.rename(columns={'#variant': 'variant'})

for truth_df in [truth1, truth2]:
    truth_df[['Chrom','pos','ref','alt']] = truth_df['variant'].str.split(':', expand=True)
    truth_df['pos'] = truth_df['pos'].astype(int)

truth_data = pd.concat([truth1, truth2], ignore_index=True)

# Process all files in the directory and create the combined matrix
process_all_files_in_directory(directory, truth_data, match_columns, output_dir)


Processed data for subdirectory: 100X
Processed data for subdirectory: 200X
Processed data for subdirectory: 300X
Processed data for subdirectory: 400X
Processed data for subdirectory: 500X
